In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-dataset/spotify_songs_en_pop.csv


# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">Lyrics Generator</p>

<img src="https://github.com/KarnikaKapoor/Files/blob/main/Pink%20and%20White%20Geometric%20Marketing%20Presentation.gif?raw=true">

<p style="font-family:newtimeroman;font-size:120%;color:#444160;">In this project, I will be building a model to generate text. My goal is to build a song lyrics generator to explore the "creative" side of the Recurrent Neural Networks(RNN). RNN Text generator is one of my most desired to-do projects. I am finally checking this one off my to-do list. So yeyyy! </p> 

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">TABLE OF CONTENTS</p>   
    
* [1. IMPORTING LIBRARIES](#1)
    
* [2. LOADING DATA](#2)
    
* [3. DATA EXPLORATION](#3)  
    
* [4. DATA PREPREPROCESSING](#4)  
    
* [5. MODEL BUILDING](#5) 
      
* [6. EVALUATING MODELS](#6)
    
* [7. CONCLUSION](#7)
    
* [8. END](#8)


<a id="1"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">IMPORTING LIBRARIES</p>


In [22]:
#importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string, os
import nltk
import re
import keras
import random
import io
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adamax
import sys
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image, ImageDraw, ImageFont
import gc
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")


<a id="2"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">LOADING DATA</p>
For this project, I have prepared a dataset of song lyrics. Let's load it and have a look.

In [23]:
data = pd.read_csv("../input/spotify-dataset/spotify_songs_en_pop.csv", nrows=1000)['lyrics']
data.head()

data, test_data = train_test_split(data, test_size=50)

In [24]:
testCorp = []

for listitem in test_data:
    testCorp.append(listitem.lower())

In [25]:
to_remove = ['*', '_', '~', '¡', '«', '´', '¿', 'à',
             'á', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ú', 'ü', 'а', 'в', 'д', 'е', 'з',
             'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'ы', 'ь',
             'ю', 'я', '\u2005', '\u200a', '\u200b', '\u3000', '、', '「', '」', 'ぁ', 'あ',
             'い', 'う', 'え', 'か', 'が', 'き', 'ぎ', 'く', 'け', 'げ', 'こ', 'さ', 'し', 'じ',
             'す', 'せ', 'そ', 'た', 'だ', 'ち', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な',
             'に', 'ぬ', 'ね', 'の', 'は', 'び', 'ふ', 'ぶ', 'へ', 'べ', 'ほ', 'ま', 'み', 'む',
             'め', 'も', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'イ', 'ウ', 'カ', 
             'キ', 'ギ', 'ク', 'シ', 'ス', 'タ', 'ダ', 'ッ', 'ツ', 'テ', 'ト', 'ネ', 'ノ', 'ビ', 
             'ポ', 'ミ', 'メ', 'モ', 'ヤ', 'ョ', 'リ', 'ル', 'レ', 'ン', 'ー', '一', '上', '世', 
             '中', '乗', '事', '人', '仆', '今', '他', '代', '体', '何', '侵', '俺', '像', '僕', 
             '償', '先', '光', '全', '共', '分', '切', '別', '前', '動', '去', '口', '叫', '名', 
             '向', '君', '呼', '咲', '嘘', '在', '壁', '壊', '声', '変', '夜', '夢', '大', '孤',
             '宿', '対', '尽', '巡', '巻', '序', '度', '弱', '張', '強', '影', '待', '後', '心',
             '恍', '恐', '悟', '悪', '情', '惚', '想', '愛', '感', '戻', '手', '抜', '拓', '指', 
             '振', '探', '揺', '支', '教', '日', '明', '昔', '昙', '時', '書', '最', '未', '来', 
             '極', '歓', '止', '正', '歩', '比', '気', '泣', '溺', '無', '特', '独', '珠', '現',
             '理', '由', '界', '痛', '白', '目', '真', '着', '瞳', '知', '破', '磨', '私', '突',
             '章', '笑', '筋', '続', '置', '肥', '胸', '自', '良', '薇', '薔', '蝕', '行', '衝', 
             '裸', '襲', '見', '覚', '见', '触', '言', '誇', '誰', '越', '踠', '身', '輝', '过',
             '返', '迷', '進', '過', '配', '重', '間', '闇', '限', '頭', '食', '高', '鳴', '갑',
             '개', '걱', '것', '겐', '겠', '견', '결', '곁', '곡', '곳', '과', '광', '교', '국',
             '군', '귀', '글', '급', '긋', '깊', '깐', '깜', '깝', '꺾', '께', '껴', '꼬', '꼭',
             '꼴', '꽃', '꽉', '꽤', '꿰', '끊', '끽', '남', '낮', '냥', '넓', '녁', '녔', '녕',
             '놀', '높', '놓', '놔', '놨', '뉴', '능', '닐', '님', '답', '당', '닿', '덜', '덩',
             '델', '독', '동', '된', '될', '둑', '둘', '둡', '뒤', '드', '득', '듭', '딱', '땀',
             '땡', '떠', '떤', '떨', '떻', '떼', '뚜', '뚫', '뜨', '뜻', '띄', '락', '럭', '럼',
             '럽', '럿', '렘', '력', '렵', '렸', '록', '론', '롭', '루', '릇', '릭', '릴', '림',
             '릿', '막', '많', '맘', '맛', '망', '맞', '맡', '맣', '매', '맨', '맴', '먼', '멀',
             '며', '명', '몇', '목', '몸', '문', '민', '밀', '밖', '발', '밝', '밟', '백', '벌',
             '베', '벼', '볍', '복', '본', '볼', '봉', '붉', '뷰', '빤', '빨', '뻑', '뻥', '뿌',
             '뿜', '쁘', '살', '삶', '석', '설', '섬', '섭', '섹', '셀', '셋', '소', '손', '솝', 
             '송', '숑', '순', '숴', '쉬', '쉽', '쉿', '슈', '스', '슨', '슴', '습', '식', '실',
             '싫', '십', '쌔', '쌤', '쏟', '쏴', '쑤', '쓰', '씩', '씹', '앉', '알', '았', '앞',
             '약', '양', '얗', '언', '엇', '엔', '열', '엽', '였', '영', '옆', '옇', '예', '옥',
             '외', '요', '용', '운', '울', '웃', '웅', '원', '윈', '유', '으', '익', '작', '잘', 
             '잠', '잡', '재', '잭', '적', '절', '젊', '점', '조', '존', '죽', '준', '줌', '줘', 
             '줬', '쥐', '증', '짓', '짝', '짧', '쩌', '찍', '착', '참', '창', '찾', '채', '챘', 
             '천', '철', '청', '쳐', '쳤', '초', '추', '축', '출', '춤', '춰', '췬', '칙', '카', 
             '커', '켜', '콘', '콤', '콱', '쿵', '크', '큰', '큼', '킬', '타', '탁', '탓', '탬', 
             '텐', '텝', '토', '통', '툭', '툰', '툴', '트', '특', '틀', '팅', '파', '팔', '패',
             '퍼', '펴', '편', '펼', '폈', '평', '포', '폼', '표', '품', '프', '플', '피', '필',
             '학', '할', '항', '핵', '행', '향', '험', '헤', '헷', '혀', '현', '호', '혹', '혼',
             '화', '확', '환', '황', '회', '효', '후', '훅', '훔', '흔', '흘', '히', '힘', 'ﬁ', 
             '\ufeff', '\xa0', 'É', 'ä', 'å', 'ç', 'ì', 'î', 'ò', 'ô', 'ö', 'ù', 'А', 'П',
             '가', '각', '간', '갈', '감', '같', '걍', '거', '건', '걷', '걸', '겁', '게', '겨', 
             '경', '계', '고', '곧', '골', '공', '관', '구', '굴', '궁', '그', '극', '근', '금', 
             '기', '긴', '길', '까', '깨', '꺼', '꾸', '꿈', '끄', '끈', '끌', '끔', '끝', '끼', 
             '낌', '나', '난', '날', '내', '냐', '너', '넌', '널', '넘', '네', '노', '논', '누', 
             '눈', '느', '는', '늘', '니', '닌', '다', '단', '달', '대', '더', '덕', '던', '데', 
             '도', '돌', '돼', '되', '두', '뒷', '든', '듣', '들', '듬', '듯', '디', '딜', '따', 
             '때', '떡', '또', '똑', '뛰', '라', '란', '람', '랑', '래', '랠', '러', '런', '렇', 
             '레', '려', '례', '로', '르', '른', '를', '름', '릎', '리', '린', '마', '만', '말', 
             '머', '먹', '멈', '멋', '멘', '면', '모', '몰', '못', '무', '묻', '물', '뭐', '뭔', 
             '뭘', '미', '믿', '바', '반', '받', '밤', '방', '배', '뱉', '버', '번', '범', '법', 
             '벗', '벽', '변', '별', '보', '봐', '부', '분', '불', '비', '빌', '빛', '빠', '뻔',
             '뿐', '뿔', '삐', '사', '상', '새', '생', '서', '선', '성', '세', '셔', '속', '쇠',
             '수', '숨', '슬', '시', '신', '심', '싶', '써', '썰', '아', '악', '안', '않', '애', 
             '앨', '야', '어', '억', '얼', '엄', '없', '었', '에', '여', '엮', '오', '온', '올', 
             '와', '완', '왔', '왜', '왠', '욕', '우', '움', '워', '위', '은', '을', '음', '의',
             '이', '인', '일', '임', '입', '있', '잊', '자', '잖', '장', '저', '전', '정', '제',
             '젠', '져', '좀', '좋', '주', '줄', '중', '즌', '즐', '지', '직', '진', '질', '짐', 
             '짜', '째', '쩜', '찌', '찔', '차', '처', '척', '체', '춘', '충', '취', '치', '친', 
             '칠', '침', '칵', '코', '콜', '키', '탈', '태', '터', '테', '투', '티', '팬', '펄', 
             '풀', '하', '한', '함', '해', '했', '허', '홀', '활', '흐', '힐', '々', 'お', 'ぐ', 
             'ば', 'ゃ', 'ア', 'ド', 'ブ', 'ム', '仲', '信', '値', '入', '受', '合', '味', '回', 
             '図', '場', '届', '平', '広', '意', '憧', '戦', '所', '拠', '描', '方', '望', '期',
             '消', '眺', '確', '空', '線', '証', '誓', '論', '起', '超', '逃', '違', '遠', '開',
             '集', '頂', '願', '駆', '鼓', '권', '넣', '둬', '뜰', '렴', '료', '밍', '션', '솔',
             '쓸', '윤', '응', '죄', '집', '텨', '픈', 'ŏ']

for i in range(len(testCorp)):
    for symbol in to_remove:
        testCorp[i] = testCorp[i].replace(symbol," ")

In [26]:
import csv

with open('/kaggle/working/baseline.csv', 'w+') as f:
    writer = csv.writer(f)

    for title in testCorp:
        writer.writerow([title])

<a id="3"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">DATA EXPLORATION</p>

**In this section, I will be:**
* Exploring the various artists in data
* Explore the number of songs and their corresponding information
* Explore the various words in lyrics via wordcloud 

In [27]:
#Printing the names of artists in the lyrics data
# print("Artists in the data:\n",data.track_artist.value_counts()) 

In [28]:
#Printing the size of dataset
# print("Size of Dataset:",data.shape)

So I have a total of 745 songs

**I will do a little feature engineering to extract more information on the songs such as:**
* Number of characters
* Number of words
* Number of lines

In [29]:
#Adding a column of numbers of Characters,words and sentences in each msg
# data["No_of_Characters"] = data["lyrics"].apply(len)
# data["No_of_Words"]=data.apply(lambda row: nltk.word_tokenize(row["lyrics"]), axis=1).apply(len)
# data["No_of_Lines"] = data["lyrics"].str.split('\n').apply(len)
# data.describe()

To be noted: On average our songs have 1400-ish characters

In [30]:
#Plotting the comparative song lengths for various artists
# plt.figure(figsize=(15,15))
# ax = sns.pairplot(data, hue="track_artist", palette="plasma")

In [31]:
# Generate a word cloud image
# stopwords = set(STOPWORDS)
# wordcloud = WordCloud(stopwords=stopwords, background_color="#444160",colormap="Purples", max_words=800).generate(" ".join(data["lyrics"]))
# plt.figure(figsize=(12,12))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.show()

*Unsurprisingly one of the most frequent words in songs is "Love".*

Next let us look at the lyrics of song No 42 in the lyrics. Why 42 you may ask! Well! because it is the answer to the Ultimate Question of Life, the Universe, and Everything. 🐬 

In [32]:
# #A function to disply the song in an asthetically pleasing way! lol
# def My_song(song):
#     img = Image.open("../input/image-for-notebook/Pink and White Geometric Marketing Presentation (1).png")
#     Text_on_image = ImageDraw.Draw(img)
#     myFont = ImageFont.truetype("../input/font-style/DancingScript-VariableFont_wght.ttf", 45)
#     Text_on_image.text((620,90), song, font=myFont, fill =(255, 255, 255))
#     return img    
# #Having a look at the first 500 charachters of a random song lyrics
# My_song(data.lyrics[42][:500])

The generation of text with RNN involves the following workflow.  

<p style="background-color:#B3C5E3;font-family:newtimeroman;color:#444160;text-align:center;font-size:120%;">Loading Data ➡️ Preprocessing ➡️ Building Mapping Dictionary ➡️ Building Model ➡️ Generating Text</p>

As I have loaded and explored the data,  I will proceed further by pre-processing the text.  


<a id="4"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;color:#444160;font-size:150%;text-align:center;border-radius:20px 60px;">DATA PREPROCESSING</p>

**In this section, I will be performing the following:**

**Creating a Corpus of Lyrics text:** For the model, we need a sequence of the text string. I am creating a Corpus out of Lyrics column. 

**Removing the unrequired characters that may have sneaked in my text corpus:** The data cleaning process for NLP is crucial preprocessing. To do that, I look into the Corpus to check for what this Corpus is comprised of. That is, all the unique symbols present. After examining the Corpus, I will be eliminating any foreign language or irrelevant symbols from the Corpus. 

**Creating a dictionary to map characters and their indices:** The computer doesn’t understand the text. For the computer, the text is just a cluster of symbols. It works with numbers. So we create a dictionary to map each unique character in our Corpus to a number and vice versa. This will be used to encode and decode the information going in and getting out of the RNN

**Splitting the corpus into smaller sentences of equal length:** Encoding and splitting the corpus into smaller sequences of equal length: At this point, Corpus contain only intended characters (i.e, lower cap English alphabets, Numbers and a few punctuations). We will encode this corpus and create small sequences of equal lengths of features and the corresponding targets. Each feature and target will contain the mapped index in the dictionary of the unique characters they signify. 

The labels are then resized and normalized. Whereas the targets are one-hot encoded. Ready to be sent to the RNN for the training, but before that let us built the RNN model. 

**Creating a Corpus**

In [33]:
#Lining up all the lyrics to create corpus
Corpus =''
for listitem in data:
    Corpus += listitem
    
Corpus = Corpus.lower() #converting all alphabets to lowecase 
del data
gc.collect()

# print("Number of unique characters:", len(set(Corpus)))

176

The total number of unique characters present in the Corpus clearly shows, that some of the foreign language scripts have sneaked in. I will take a look at all the characters present. I will then remove the unrequired characters. 

In [34]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: [' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '«', '´', '¿', 'à', 'á', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ú', 'ü', 'а', 'в', 'д', 'е', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'ы', 'ь', 'ю', 'я', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '‚', '“', '”', '…', '가', '각', '간', '갈', '감', '같', '걍', '거', '건', '걷', '걸', '겁', '게', '겨', '경', '계', '고', '곧', '골', '공', '관', '구', '굴', '궁', '권', '그', '극', '근', '금', '기', '긴', '길', '까', '깨', '꺼', '꾸', '꿈', '끄', '끈', '끌', '끔', '끝', '끼', '낌', '나', '난', '날', '내', '냐', '너', '넌', '널', '넘', '네', '노', '논', '누', '눈', '느', '는', '늘', '니', '닌', '다', '단', '달', '대', '더', '덕', '던', '데', '도', '돌', '돼', '되', '두', '뒷', '든', '듣', '들', '듬', '듯', '디', '딜', '따', '때', '떡', '또', '똑', '뛰', '

In [35]:
#Keeping only a limited set of characters. 
to_remove = ['*', '_', '~', '¡', '«', '´', '¿', 'à',
             'á', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ú', 'ü', 'а', 'в', 'д', 'е', 'з',
             'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'ы', 'ь',
             'ю', 'я', '\u2005', '\u200a', '\u200b', '\u3000', '、', '「', '」', 'ぁ', 'あ',
             'い', 'う', 'え', 'か', 'が', 'き', 'ぎ', 'く', 'け', 'げ', 'こ', 'さ', 'し', 'じ',
             'す', 'せ', 'そ', 'た', 'だ', 'ち', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な',
             'に', 'ぬ', 'ね', 'の', 'は', 'び', 'ふ', 'ぶ', 'へ', 'べ', 'ほ', 'ま', 'み', 'む',
             'め', 'も', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'イ', 'ウ', 'カ', 
             'キ', 'ギ', 'ク', 'シ', 'ス', 'タ', 'ダ', 'ッ', 'ツ', 'テ', 'ト', 'ネ', 'ノ', 'ビ', 
             'ポ', 'ミ', 'メ', 'モ', 'ヤ', 'ョ', 'リ', 'ル', 'レ', 'ン', 'ー', '一', '上', '世', 
             '中', '乗', '事', '人', '仆', '今', '他', '代', '体', '何', '侵', '俺', '像', '僕', 
             '償', '先', '光', '全', '共', '分', '切', '別', '前', '動', '去', '口', '叫', '名', 
             '向', '君', '呼', '咲', '嘘', '在', '壁', '壊', '声', '変', '夜', '夢', '大', '孤',
             '宿', '対', '尽', '巡', '巻', '序', '度', '弱', '張', '強', '影', '待', '後', '心',
             '恍', '恐', '悟', '悪', '情', '惚', '想', '愛', '感', '戻', '手', '抜', '拓', '指', 
             '振', '探', '揺', '支', '教', '日', '明', '昔', '昙', '時', '書', '最', '未', '来', 
             '極', '歓', '止', '正', '歩', '比', '気', '泣', '溺', '無', '特', '独', '珠', '現',
             '理', '由', '界', '痛', '白', '目', '真', '着', '瞳', '知', '破', '磨', '私', '突',
             '章', '笑', '筋', '続', '置', '肥', '胸', '自', '良', '薇', '薔', '蝕', '行', '衝', 
             '裸', '襲', '見', '覚', '见', '触', '言', '誇', '誰', '越', '踠', '身', '輝', '过',
             '返', '迷', '進', '過', '配', '重', '間', '闇', '限', '頭', '食', '高', '鳴', '갑',
             '개', '걱', '것', '겐', '겠', '견', '결', '곁', '곡', '곳', '과', '광', '교', '국',
             '군', '귀', '글', '급', '긋', '깊', '깐', '깜', '깝', '꺾', '께', '껴', '꼬', '꼭',
             '꼴', '꽃', '꽉', '꽤', '꿰', '끊', '끽', '남', '낮', '냥', '넓', '녁', '녔', '녕',
             '놀', '높', '놓', '놔', '놨', '뉴', '능', '닐', '님', '답', '당', '닿', '덜', '덩',
             '델', '독', '동', '된', '될', '둑', '둘', '둡', '뒤', '드', '득', '듭', '딱', '땀',
             '땡', '떠', '떤', '떨', '떻', '떼', '뚜', '뚫', '뜨', '뜻', '띄', '락', '럭', '럼',
             '럽', '럿', '렘', '력', '렵', '렸', '록', '론', '롭', '루', '릇', '릭', '릴', '림',
             '릿', '막', '많', '맘', '맛', '망', '맞', '맡', '맣', '매', '맨', '맴', '먼', '멀',
             '며', '명', '몇', '목', '몸', '문', '민', '밀', '밖', '발', '밝', '밟', '백', '벌',
             '베', '벼', '볍', '복', '본', '볼', '봉', '붉', '뷰', '빤', '빨', '뻑', '뻥', '뿌',
             '뿜', '쁘', '살', '삶', '석', '설', '섬', '섭', '섹', '셀', '셋', '소', '손', '솝', 
             '송', '숑', '순', '숴', '쉬', '쉽', '쉿', '슈', '스', '슨', '슴', '습', '식', '실',
             '싫', '십', '쌔', '쌤', '쏟', '쏴', '쑤', '쓰', '씩', '씹', '앉', '알', '았', '앞',
             '약', '양', '얗', '언', '엇', '엔', '열', '엽', '였', '영', '옆', '옇', '예', '옥',
             '외', '요', '용', '운', '울', '웃', '웅', '원', '윈', '유', '으', '익', '작', '잘', 
             '잠', '잡', '재', '잭', '적', '절', '젊', '점', '조', '존', '죽', '준', '줌', '줘', 
             '줬', '쥐', '증', '짓', '짝', '짧', '쩌', '찍', '착', '참', '창', '찾', '채', '챘', 
             '천', '철', '청', '쳐', '쳤', '초', '추', '축', '출', '춤', '춰', '췬', '칙', '카', 
             '커', '켜', '콘', '콤', '콱', '쿵', '크', '큰', '큼', '킬', '타', '탁', '탓', '탬', 
             '텐', '텝', '토', '통', '툭', '툰', '툴', '트', '특', '틀', '팅', '파', '팔', '패',
             '퍼', '펴', '편', '펼', '폈', '평', '포', '폼', '표', '품', '프', '플', '피', '필',
             '학', '할', '항', '핵', '행', '향', '험', '헤', '헷', '혀', '현', '호', '혹', '혼',
             '화', '확', '환', '황', '회', '효', '후', '훅', '훔', '흔', '흘', '히', '힘', 'ﬁ', 
             '\ufeff', '\xa0', 'É', 'ä', 'å', 'ç', 'ì', 'î', 'ò', 'ô', 'ö', 'ù', 'А', 'П',
             '가', '각', '간', '갈', '감', '같', '걍', '거', '건', '걷', '걸', '겁', '게', '겨', 
             '경', '계', '고', '곧', '골', '공', '관', '구', '굴', '궁', '그', '극', '근', '금', 
             '기', '긴', '길', '까', '깨', '꺼', '꾸', '꿈', '끄', '끈', '끌', '끔', '끝', '끼', 
             '낌', '나', '난', '날', '내', '냐', '너', '넌', '널', '넘', '네', '노', '논', '누', 
             '눈', '느', '는', '늘', '니', '닌', '다', '단', '달', '대', '더', '덕', '던', '데', 
             '도', '돌', '돼', '되', '두', '뒷', '든', '듣', '들', '듬', '듯', '디', '딜', '따', 
             '때', '떡', '또', '똑', '뛰', '라', '란', '람', '랑', '래', '랠', '러', '런', '렇', 
             '레', '려', '례', '로', '르', '른', '를', '름', '릎', '리', '린', '마', '만', '말', 
             '머', '먹', '멈', '멋', '멘', '면', '모', '몰', '못', '무', '묻', '물', '뭐', '뭔', 
             '뭘', '미', '믿', '바', '반', '받', '밤', '방', '배', '뱉', '버', '번', '범', '법', 
             '벗', '벽', '변', '별', '보', '봐', '부', '분', '불', '비', '빌', '빛', '빠', '뻔',
             '뿐', '뿔', '삐', '사', '상', '새', '생', '서', '선', '성', '세', '셔', '속', '쇠',
             '수', '숨', '슬', '시', '신', '심', '싶', '써', '썰', '아', '악', '안', '않', '애', 
             '앨', '야', '어', '억', '얼', '엄', '없', '었', '에', '여', '엮', '오', '온', '올', 
             '와', '완', '왔', '왜', '왠', '욕', '우', '움', '워', '위', '은', '을', '음', '의',
             '이', '인', '일', '임', '입', '있', '잊', '자', '잖', '장', '저', '전', '정', '제',
             '젠', '져', '좀', '좋', '주', '줄', '중', '즌', '즐', '지', '직', '진', '질', '짐', 
             '짜', '째', '쩜', '찌', '찔', '차', '처', '척', '체', '춘', '충', '취', '치', '친', 
             '칠', '침', '칵', '코', '콜', '키', '탈', '태', '터', '테', '투', '티', '팬', '펄', 
             '풀', '하', '한', '함', '해', '했', '허', '홀', '활', '흐', '힐', '々', 'お', 'ぐ', 
             'ば', 'ゃ', 'ア', 'ド', 'ブ', 'ム', '仲', '信', '値', '入', '受', '合', '味', '回', 
             '図', '場', '届', '平', '広', '意', '憧', '戦', '所', '拠', '描', '方', '望', '期',
             '消', '眺', '確', '空', '線', '証', '誓', '論', '起', '超', '逃', '違', '遠', '開',
             '集', '頂', '願', '駆', '鼓', '권', '넣', '둬', '뜰', '렴', '료', '밍', '션', '솔',
             '쓸', '윤', '응', '죄', '집', '텨', '픈', 'ŏ']
for symbol in to_remove:
    Corpus = Corpus.replace(symbol," ")
    
del to_remove
gc.collect()

#Corpus = re.sub("[^A-Za-z0-9'\.\n]","",Corpus) Alterneativly could be used but I want to pick and chose (:

0

In [36]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: [' ', '!', '"', '#', '$', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '–', '—', '‘', '‚', '“', '”', '…']


**Creating a list of sorted unique characters**

In [37]:
# Storing all the unique characters present in my corpus to bult a mapping dic. 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)
del symb
gc.collect()

Total number of characters: 2226491
Number of unique characters: 59


44

In [38]:
#Splitting the Corpus in equal length of strings and output target
length = 40
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
    
L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

Total number of sequences in the Corpus: 2226451


**Encoding the Labels and Targets**

In [39]:
# reshape X and normalize
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(L_symb)
del features, L_datapoints, L_corpus
gc.collect()

# one hot encode the output variable
y = np_utils.to_categorical(targets)
del targets
gc.collect()

0

<a id="5"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;font-size:150%;color:#444160;text-align:center;border-radius:20px 60px;">MODEL BUILDING</p>


Recurrent Neural Networks are pretty popular with generating text. In this project, I will be using a LSTM Model, an improved version of a standard recurrent neural network

**Following steps are involved in the model building**

* Initialising the Model
* Defining by adding layers
* Compiling the Model
* Training the Model

**Building the Model**

In [40]:
#Initialising the Model
model = Sequential()
#Adding layers
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1],f activation='softmax'))
#Compiling the model for training  
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

#Model's Summary               
model.summary()

SyntaxError: invalid syntax (<ipython-input-40-a9a31df2e36b>, line 5)

In [ ]:
#Training the Model
history = model.fit(X, y, batch_size=128, epochs=100)

In [ ]:
from keras.models import load_model
#To be used later; I am saving the model 
model.save("Lyrics_Generator.h5")

<a id="6"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;font-size:150%;color:#444160;text-align:center;border-radius:20px 60px;">EVALUATING MODELS</p>

Now that I have my model trained on the songs lyrics let us see how it performs. I hope it creates some sensible song.

**To evaluate my model, I shall be having a look at:**
* The performance of the model via Learning Curves
* The outcome text it generates

**Plotting the learning curve for the loss function**

In [ ]:
# history_df = pd.DataFrame(history.history)
# #Plotting the learnings 

# fig = plt.figure(figsize=(15,4), facecolor="#B291B6")
# fig.suptitle("Learning Plot of Model for Loss")
# pl=sns.lineplot(data=history_df["loss"],color="#444160")
# pl.set(ylabel ="Training Loss")
# pl.set(xlabel ="Epochs")

**Generating the songs**

In [ ]:
# The function to generate text from model
def Lyrics_Generator(starter,Ch_count): #,temperature=1.0):
    generated= ""
    starter = starter 
    seed=[mapping[char] for char in starter]
    print(seed)
    generated += starter 
    # Generating new text of given length
    for i in range(Ch_count):
        seed=[mapping[char] for char in starter]
        x_pred = np.reshape(seed, (1, len(seed), 1))
        x_pred = x_pred/ float(L_symb)
        prediction = model.predict(x_pred, verbose=0)[0]  
        # Getting the index of the next most probable index
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / 1.0 
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, prediction, 1)
        index = np.argmax(prediction)
        next_char = reverse_mapping[index]  
        # Generating new text
        generated += next_char
        starter = starter[1:] + next_char
       
    return generated

Let us finally see the outcome by putting a seed in from one of my old blogposts [blogpost](https://karnikakapoor.blogspot.com/2017/04/killers-confession.html)

In [ ]:
# for i in test_data:
#     print(i)
#     print()

In [ ]:
testCorp = []

for listitem in test_data:
    testCorp.append(listitem.lower())

In [ ]:
#Keeping only a limited set of characters. 
to_remove = ['*', '_', '~', '¡', '«', '´', '¿', 'à',
             'á', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ú', 'ü', 'а', 'в', 'д', 'е', 'з',
             'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'ы', 'ь',
             'ю', 'я', '\u2005', '\u200a', '\u200b', '\u3000', '、', '「', '」', 'ぁ', 'あ',
             'い', 'う', 'え', 'か', 'が', 'き', 'ぎ', 'く', 'け', 'げ', 'こ', 'さ', 'し', 'じ',
             'す', 'せ', 'そ', 'た', 'だ', 'ち', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な',
             'に', 'ぬ', 'ね', 'の', 'は', 'び', 'ふ', 'ぶ', 'へ', 'べ', 'ほ', 'ま', 'み', 'む',
             'め', 'も', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'イ', 'ウ', 'カ', 
             'キ', 'ギ', 'ク', 'シ', 'ス', 'タ', 'ダ', 'ッ', 'ツ', 'テ', 'ト', 'ネ', 'ノ', 'ビ', 
             'ポ', 'ミ', 'メ', 'モ', 'ヤ', 'ョ', 'リ', 'ル', 'レ', 'ン', 'ー', '一', '上', '世', 
             '中', '乗', '事', '人', '仆', '今', '他', '代', '体', '何', '侵', '俺', '像', '僕', 
             '償', '先', '光', '全', '共', '分', '切', '別', '前', '動', '去', '口', '叫', '名', 
             '向', '君', '呼', '咲', '嘘', '在', '壁', '壊', '声', '変', '夜', '夢', '大', '孤',
             '宿', '対', '尽', '巡', '巻', '序', '度', '弱', '張', '強', '影', '待', '後', '心',
             '恍', '恐', '悟', '悪', '情', '惚', '想', '愛', '感', '戻', '手', '抜', '拓', '指', 
             '振', '探', '揺', '支', '教', '日', '明', '昔', '昙', '時', '書', '最', '未', '来', 
             '極', '歓', '止', '正', '歩', '比', '気', '泣', '溺', '無', '特', '独', '珠', '現',
             '理', '由', '界', '痛', '白', '目', '真', '着', '瞳', '知', '破', '磨', '私', '突',
             '章', '笑', '筋', '続', '置', '肥', '胸', '自', '良', '薇', '薔', '蝕', '行', '衝', 
             '裸', '襲', '見', '覚', '见', '触', '言', '誇', '誰', '越', '踠', '身', '輝', '过',
             '返', '迷', '進', '過', '配', '重', '間', '闇', '限', '頭', '食', '高', '鳴', '갑',
             '개', '걱', '것', '겐', '겠', '견', '결', '곁', '곡', '곳', '과', '광', '교', '국',
             '군', '귀', '글', '급', '긋', '깊', '깐', '깜', '깝', '꺾', '께', '껴', '꼬', '꼭',
             '꼴', '꽃', '꽉', '꽤', '꿰', '끊', '끽', '남', '낮', '냥', '넓', '녁', '녔', '녕',
             '놀', '높', '놓', '놔', '놨', '뉴', '능', '닐', '님', '답', '당', '닿', '덜', '덩',
             '델', '독', '동', '된', '될', '둑', '둘', '둡', '뒤', '드', '득', '듭', '딱', '땀',
             '땡', '떠', '떤', '떨', '떻', '떼', '뚜', '뚫', '뜨', '뜻', '띄', '락', '럭', '럼',
             '럽', '럿', '렘', '력', '렵', '렸', '록', '론', '롭', '루', '릇', '릭', '릴', '림',
             '릿', '막', '많', '맘', '맛', '망', '맞', '맡', '맣', '매', '맨', '맴', '먼', '멀',
             '며', '명', '몇', '목', '몸', '문', '민', '밀', '밖', '발', '밝', '밟', '백', '벌',
             '베', '벼', '볍', '복', '본', '볼', '봉', '붉', '뷰', '빤', '빨', '뻑', '뻥', '뿌',
             '뿜', '쁘', '살', '삶', '석', '설', '섬', '섭', '섹', '셀', '셋', '소', '손', '솝', 
             '송', '숑', '순', '숴', '쉬', '쉽', '쉿', '슈', '스', '슨', '슴', '습', '식', '실',
             '싫', '십', '쌔', '쌤', '쏟', '쏴', '쑤', '쓰', '씩', '씹', '앉', '알', '았', '앞',
             '약', '양', '얗', '언', '엇', '엔', '열', '엽', '였', '영', '옆', '옇', '예', '옥',
             '외', '요', '용', '운', '울', '웃', '웅', '원', '윈', '유', '으', '익', '작', '잘', 
             '잠', '잡', '재', '잭', '적', '절', '젊', '점', '조', '존', '죽', '준', '줌', '줘', 
             '줬', '쥐', '증', '짓', '짝', '짧', '쩌', '찍', '착', '참', '창', '찾', '채', '챘', 
             '천', '철', '청', '쳐', '쳤', '초', '추', '축', '출', '춤', '춰', '췬', '칙', '카', 
             '커', '켜', '콘', '콤', '콱', '쿵', '크', '큰', '큼', '킬', '타', '탁', '탓', '탬', 
             '텐', '텝', '토', '통', '툭', '툰', '툴', '트', '특', '틀', '팅', '파', '팔', '패',
             '퍼', '펴', '편', '펼', '폈', '평', '포', '폼', '표', '품', '프', '플', '피', '필',
             '학', '할', '항', '핵', '행', '향', '험', '헤', '헷', '혀', '현', '호', '혹', '혼',
             '화', '확', '환', '황', '회', '효', '후', '훅', '훔', '흔', '흘', '히', '힘', 'ﬁ', 
             '\ufeff', '\xa0', 'É', 'ä', 'å', 'ç', 'ì', 'î', 'ò', 'ô', 'ö', 'ù', 'А', 'П',
             '가', '각', '간', '갈', '감', '같', '걍', '거', '건', '걷', '걸', '겁', '게', '겨', 
             '경', '계', '고', '곧', '골', '공', '관', '구', '굴', '궁', '그', '극', '근', '금', 
             '기', '긴', '길', '까', '깨', '꺼', '꾸', '꿈', '끄', '끈', '끌', '끔', '끝', '끼', 
             '낌', '나', '난', '날', '내', '냐', '너', '넌', '널', '넘', '네', '노', '논', '누', 
             '눈', '느', '는', '늘', '니', '닌', '다', '단', '달', '대', '더', '덕', '던', '데', 
             '도', '돌', '돼', '되', '두', '뒷', '든', '듣', '들', '듬', '듯', '디', '딜', '따', 
             '때', '떡', '또', '똑', '뛰', '라', '란', '람', '랑', '래', '랠', '러', '런', '렇', 
             '레', '려', '례', '로', '르', '른', '를', '름', '릎', '리', '린', '마', '만', '말', 
             '머', '먹', '멈', '멋', '멘', '면', '모', '몰', '못', '무', '묻', '물', '뭐', '뭔', 
             '뭘', '미', '믿', '바', '반', '받', '밤', '방', '배', '뱉', '버', '번', '범', '법', 
             '벗', '벽', '변', '별', '보', '봐', '부', '분', '불', '비', '빌', '빛', '빠', '뻔',
             '뿐', '뿔', '삐', '사', '상', '새', '생', '서', '선', '성', '세', '셔', '속', '쇠',
             '수', '숨', '슬', '시', '신', '심', '싶', '써', '썰', '아', '악', '안', '않', '애', 
             '앨', '야', '어', '억', '얼', '엄', '없', '었', '에', '여', '엮', '오', '온', '올', 
             '와', '완', '왔', '왜', '왠', '욕', '우', '움', '워', '위', '은', '을', '음', '의',
             '이', '인', '일', '임', '입', '있', '잊', '자', '잖', '장', '저', '전', '정', '제',
             '젠', '져', '좀', '좋', '주', '줄', '중', '즌', '즐', '지', '직', '진', '질', '짐', 
             '짜', '째', '쩜', '찌', '찔', '차', '처', '척', '체', '춘', '충', '취', '치', '친', 
             '칠', '침', '칵', '코', '콜', '키', '탈', '태', '터', '테', '투', '티', '팬', '펄', 
             '풀', '하', '한', '함', '해', '했', '허', '홀', '활', '흐', '힐', '々', 'お', 'ぐ', 
             'ば', 'ゃ', 'ア', 'ド', 'ブ', 'ム', '仲', '信', '値', '入', '受', '合', '味', '回', 
             '図', '場', '届', '平', '広', '意', '憧', '戦', '所', '拠', '描', '方', '望', '期',
             '消', '眺', '確', '空', '線', '証', '誓', '論', '起', '超', '逃', '違', '遠', '開',
             '集', '頂', '願', '駆', '鼓', '권', '넣', '둬', '뜰', '렴', '료', '밍', '션', '솔',
             '쓸', '윤', '응', '죄', '집', '텨', '픈', 'ŏ']

for i in range(len(testCorp)):
    for symbol in to_remove:
        testCorp[i] = testCorp[i].replace(symbol," ")
    
del to_remove
gc.collect()

#Corpus = re.sub("[^A-Za-z0-9'\.\n]","",Corpus) Alterneativly could be used but I want to pick and chose (:

In [ ]:
# for title in testCorp:
#     print(title)
#     print()

In [ ]:
#Generating a song from the model
results = {}

for title in testCorp: 
    beginning = " ".join(title.split()[0:5])
    beginning = beginning.lower()
    song = Lyrics_Generator(beginning, 400)
    results[beginning] = song

In [ ]:
print(len(results))

In [ ]:
import csv # move up

with open('/kaggle/working/baseline_lyrics.csv', 'w+') as f:
    writer = csv.writer(f)
    
    header = ['Prompt', 'Generation', 'Coherence', 'Musical Quality', 'Pop Style']
    writer.writerow(header)

    for prompt, lyrics in results.items():
        writer.writerow([prompt, lyrics])

Another song generated by a seed of the lyrics of a song that's stuck in my head today. (Sunflower by Shannon Purser)

# <a id="7"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;font-size:150%;color:#444160;text-align:center;border-radius:20px 60px;">CONCLUSION</p>
<p style="font-family:newtimeroman;font-size:120%;color:#444160">On observing the output of the Lyrics Generator, it is clear that while some of the sentences might be correct, but most of the lyrics do not make sense. It does look like a song tho. The model didn't learn the meaning of the songs. However, the character-based approach is producing some legitimate words. 
To get to a song that makes better sense I may consider a transformer-based text generator, but that's for some other time.</p>

<p style="font-family:newtimeroman;font-size:120%;color:#444160;"> Neural networks amaze me all the time. There is something surreal about them that makes working with it exciting. Of course, we can peel off the layers and see the maths behind them. Get to the matrices and tensor to understand how these neurons are working. Even get the values of weights and biases and assure ourselves that this is no sorcery.  Still, when I see the result play out it is astonishing.</p>  

**Some useful resources on Text generation:**

[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

[RNN ](https://towardsdatascience.com/character-level-language-model-1439f5dd87fe)

[GTP-2 ](https://openai.com/blog/better-language-models/) 


**<span style="color:#444160;"> If you liked this Notebook, please do upvote.</span>**

**<span style="color:#444160;"> Best Wishes!</span>**

<a id="8"></a>
# <p style="background-color:#B291B6;font-family:newtimeroman;font-size:150%;color:#444160;text-align:center;border-radius:20px 60px;">END</p>